---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/28_Vasicek.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<img src="https://www.dropbox.com/scl/fi/6hwvdff7ajaafmkpmnp0o/under_construction.jpg?rlkey=3dex2dx86anniqoutwyqashnu&dl=1" alt="Under Construction" width="400"/>


In [ ]:


import numpy as np
from scipy.stats import norm

def black_call(F, K, P, sigma, T):
    d1 = (np.log(F / K) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return P * (F * norm.cdf(d1) - K * norm.cdf(d2))

def black_put(F, K, P, sigma, T):
    d1 = (np.log(F / K) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return P * (K * norm.cdf(-d2) - F * norm.cdf(-d1))

def vasicek_discount_bond_call(Underlying, K, MatDisc, sigavg, T):
    F = Underlying / MatDisc
    return black_call(F, K, MatDisc, sigavg, T)

def vasicek_discount_bond_put(Underlying, K, MatDisc, sigavg, T):
    F = Underlying / MatDisc
    return black_put(F, K, MatDisc, sigavg, T)


# Example usage
Underlying = 0.9
K = 0.9
MatDisc = 0.88
sigavg = 0.2
T = 1


print("Vasicek Discount Bond Call:", vasicek_discount_bond_call(Underlying, K, MatDisc, sigavg, T))
print("Vasicek Discount Bond Put:", vasicek_discount_bond_put(Underlying, K, MatDisc, sigavg, T))

In [ ]:


def vasicek_cap(P0, P, rbar, sigavg, N, t0, dt):
    K = 1 / (1 + rbar * dt)
    if t0 == 0:
        cap_value = P[0] * max(0, 1 / P[0] - 1 - rbar * dt)
    else:
        MatDisc = P0
        Underlying = P[0]
        cap_value = vasicek_discount_bond_put(Underlying, K, MatDisc, sigavg[0], t0)
    
    for i in range(1, N):
        MatDisc = P[i - 1]
        Underlying = P[i]
        mat = t0 + i * dt
        cap_value += vasicek_discount_bond_put(Underlying, K, MatDisc, sigavg[i], mat)
    
    return (1 + rbar * dt) * cap_value

# Example

P0 = 0.95
P = [0.92, 0.89, 0.85, 0.80]
rbar = 0.03
N = 4
t0 = 0.5
dt = 0.5

print("Vasicek Cap:", vasicek_cap(P0, P, rbar, [0.2, 0.18, 0.15, 0.12], N, t0, dt))

In [ ]:


def hwa(sigma, kappa, tau):
    return -sigma ** 2 * (2 * kappa * tau - np.exp(-2 * kappa * tau) + 4 * np.exp(-kappa * tau) - 3) / (4 * kappa ** 3)

def hwb(kappa, tau):
    return (1 - np.exp(-kappa * tau)) / kappa

def hwsigavg(sigma, kappa, T, u):
    return max((sigma * (np.exp(-kappa * T) - np.exp(-kappa * u)) / kappa) * np.sqrt((np.exp(2 * kappa * T) - 1) / (2 * kappa * T)),0.001)

def hw_coup_bond(Coup, N, t1, dt, Cal, r, sigma, kappa, T):
    x = 0
    for i in range(N - 1):
        tau = t1 + i * dt - T
        a = hwa(sigma, kappa, tau)
        b = hwb(kappa, tau)
        x += Coup * Cal[i] * np.exp(-a - b * r)
    
    tau = t1 + (N - 1) * dt - T
    a = hwa(sigma, kappa, tau)
    b = hwb(kappa, tau)
    x += (1 + Coup) * Cal[N - 1] * np.exp(-a - b * r)
    
    return x

def hw_coup_bond_call(Coup, N, t1, dt, K, MatDisc, P, r, sigma, kappa, T):
    tol = 1e-8
    Cal = np.zeros(N)
    
    aT = hwa(sigma, kappa, T)
    bT = hwb(kappa, T)
    for i in range(N):
        tau = t1 + i * dt
        a = hwa(sigma, kappa, tau)
        b = hwb(kappa, tau)
        Cal[i] = np.exp(a - aT + (b - bT) * r) * P[i] / MatDisc
    
    lower = 0
    flower = hw_coup_bond(Coup, N, t1, dt, Cal, lower, sigma, kappa, T) - K
    while flower < 0:
        lower -= 1
        flower = hw_coup_bond(Coup, N, t1, dt, Cal, lower, sigma, kappa, T) - K
    
    upper = 1
    fupper = hw_coup_bond(Coup, N, t1, dt, Cal, upper, sigma, kappa, T) - K
    while fupper > 0:
        upper += 1
        fupper = hw_coup_bond(Coup, N, t1, dt, Cal, upper, sigma, kappa, T) - K
    
    guess = 0.5 * (lower + upper)
    fguess = hw_coup_bond(Coup, N, t1, dt, Cal, guess, sigma, kappa, T) - K
    while upper - lower > tol:
        if fupper * fguess < 0:
            lower = guess
            flower = fguess
        else:
            upper = guess
            fupper = fguess
        guess = 0.5 * (lower + upper)
        fguess = hw_coup_bond(Coup, N, t1, dt, Cal, guess, sigma, kappa, T) - K
    
    rstar = guess
    x = 0
    
    for i in range(N - 1):
        tau = t1 + i * dt - T
        a = hwa(sigma, kappa, tau)
        b = hwb(kappa, tau)
        strike = Cal[i] * np.exp(-a - b * rstar)
        sigavg = hwsigavg(sigma, kappa, T, T + tau)
        x += Coup * vasicek_discount_bond_call(P[i], strike, MatDisc, sigavg, T)
    
    tau = t1 + (N - 1) * dt - T
    a = hwa(sigma, kappa, tau)
    b = hwb(kappa, tau)
    strike = Cal[N - 1] * np.exp(-a - b * rstar)
    sigavg = hwsigavg(sigma, kappa, T, T + tau)
    x += (1 + Coup) * vasicek_discount_bond_call(P[N - 1], strike, MatDisc, sigavg, T)
    
    return x

# Example usage

Underlying = 0.95
K = 0.9
MatDisc = 0.88
sigavg = 0.2
T = 1

P0 = 0.95
P = [0.92, 0.89, 0.85, 0.80]
rbar = 0.03
N = 4
t0 = 0.5
dt = 0.5

Coup = 0.03
r = 0.02
sigma = 0.1
kappa = 0.1


print("HW Coupon Bond Call:", hw_coup_bond_call(Coup, N, t0, dt, K, MatDisc, P, r, sigma, kappa, T))